# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 08

### Elementos do Grupo

Número: 54329   Nome: David da Costa Correia    
Número: 56906   Nome: Miguel Castro  
Número: 56922   Nome: João Leal

<hr>

## Quadrados Latinos

### Representação de variáveis, domínios, vizinhos e restrições

#### Variáveis

#### Domínios

#### Restrições

### Formulação do Problema

In [1]:
def grid_neighbors(cell, grid):
    '''Returns the grid neighbors of a cell, i.e. all the cells 
    that share a row or column with the given cell.'''
    x, y = cell[0], cell[1]
    return [cel for cel in grid if ((cel[0] == x) or (cel[1] == y)) and (cel != cell)]

from csp import *

def latin_square(dim, numbers={}):

    variables = [(x,y) for x in range(1,dim+1) for y in range(1,dim+1)]

    domains = {v:list(range(1,dim+1)) for v in variables}

    for cell,value in numbers.items():
        if value > dim:
            raise 'Erro: valor atribuído inválido'
        else:
            domains[cell] = [value]

    neighbors = {v:grid_neighbors(v, variables) for v in variables}

    def constraints(X, a, Y, b):
        return a != b
    
    return CSP(variables, domains, neighbors, constraints)

### Visualização do problema

In [24]:
def force_ordered_board(dim, board):
    '''Forces the natural order of elements in the board.
    Example: {(1,1):N, (2,1):N, ... (i,j):N}'''
    template = [(x,y) for y in range(1,dim+1) for x in range(1,dim+1)] # [(1,1), (2,1), ...]
    return {cell:board[cell] for cell in template}

def latin_square_output(dim, board):
    '''Prints a latin square board.'''
    board = force_ordered_board(dim, board)
    i = 0
    output = ''
    for value in board.values():
        if i < dim: # the output must be as long as dim
            output = output + str(value) + ' '
            i += 1
        else: # print output row and start a new one
            print(output)
            output = str(value) + ' '
            i = 1
    print(output) # print the last row

def display(dim, board, neqs=None):
    '''Displays a latin square or futoshiki board 
    wheter neqs is not given or given, respectively.
    It also can display either unsolved or solved problems.'''
    if isinstance(board, dict): # solved problem
        if not neqs:
            latin_square_output(dim, board)
        else: futoshiki_output(dim, board, neqs)
    else: # unsolved problem
        board = board.domains.copy()

        for cell,domain in board.items():
            if len(domain) > 1: # i.e. there were no pre-existing given numbers 
                board[cell] = '-'
            else:
                board[cell] = domain[0] # domain[0] is the pre-existing given number

        if not neqs:
            latin_square_output(dim, board)
        else: futoshiki_output(dim, board, neqs)

### Instâncias e Testes

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

In [23]:
tls = latin_square(10)#, board={(2,2):3})

Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

In [31]:
import timeit

#tls_ac3 = AC3(tls) 

# Sem inferência

start = timeit.default_timer()
ls_no_inference = backtracking_search(tls)
stop = timeit.default_timer()
time = stop-start

print("Sem inferência:")
display(10, ls_no_inference)
print(time)

# Com inferência Forward-Checking

start = timeit.default_timer()
ls_fc = backtracking_search(tls, inference=forward_checking)
stop = timeit.default_timer()
time = stop-start

print("\nCom inferência forward-checking:")
display(10, ls_fc)
print(time)

# Com inferência MAC

start = timeit.default_timer()
ls_mac = backtracking_search(tls, inference=mac)
stop = timeit.default_timer()
time = stop-start

print("\nCom inferência MAC:")
display(10, ls_mac)
print(time)

# Heurística

start = timeit.default_timer()
ls_heu = backtracking_search(tls, select_unassigned_variable=mrv)
stop = timeit.default_timer()
time = stop-start

print("\nCom heurística:")
display(10, ls_heu)
print(time)

Sem inferência:
1 2 3 4 5 6 7 8 9 10 
2 1 4 3 6 5 8 7 10 9 
3 4 1 2 7 8 9 10 5 6 
4 3 2 1 8 7 10 9 6 5 
5 6 7 8 9 10 4 1 2 3 
6 5 8 7 10 9 1 3 4 2 
7 8 9 10 3 2 5 6 1 4 
8 7 10 9 2 4 6 5 3 1 
9 10 6 5 4 1 3 2 7 8 
10 9 5 6 1 3 2 4 8 7 
0.002010799999993651

Com inferência forward-checking:
1 2 3 4 5 6 7 8 9 10 
2 1 4 3 6 5 8 7 10 9 
3 4 1 2 7 8 9 10 5 6 
4 3 2 1 8 7 10 9 6 5 
5 6 7 8 9 10 4 1 2 3 
6 5 8 7 10 9 1 3 4 2 
7 8 9 10 3 2 5 6 1 4 
8 7 10 9 2 4 6 5 3 1 
9 10 6 5 4 1 3 2 7 8 
10 9 5 6 1 3 2 4 8 7 
0.0025580000000218206

Com inferência MAC:
1 2 3 4 5 6 7 8 9 10 
2 1 4 3 6 5 8 7 10 9 
3 4 1 2 7 8 9 10 5 6 
4 3 2 1 8 7 10 9 6 5 
5 6 7 8 9 10 4 1 2 3 
6 5 8 7 10 9 1 3 4 2 
7 8 9 10 3 2 5 6 1 4 
8 7 10 9 2 4 6 5 3 1 
9 10 6 5 4 1 3 2 7 8 
10 9 5 6 1 3 2 4 8 7 
0.003725499999973181

Com heurística:
1 2 3 4 5 6 7 8 9 10 
2 1 4 3 6 5 8 7 10 9 
3 4 1 2 7 8 9 10 5 6 
4 3 2 1 8 7 10 9 6 5 
5 6 7 8 9 10 4 1 2 3 
6 5 8 7 10 9 1 3 4 2 
7 8 9 10 3 2 5 6 1 4 
8 7 10 9 2 4 6 5 3 1 
9 10 6 5 4 1

<hr>

## Futoshiki

### Representação de variáveis, domínios, vizinhos e restrições

#### Variáveis

#### Domínios

#### Restrições

### Formulação do Problema

In [59]:
def gt(a,b):
    return a > b

def lt(a,b):
    return a < b

def diff(a,b):
    return a != b

def futoshiki(dim, numbers={}, neqs=[]):
    variables = [(x,y) for x in range(1,dim+1) for y in range(1,dim+1)]

    domains = {v:list(range(1,dim+1)) for v in variables}

    for cell,value in numbers.items():
        if value > dim:
            raise 'Erro: valor atribuído inválido'
        else:
            domains[cell] = [value]

    neighbors = {v:grid_neighbors(v, variables) for v in variables}

    def constraints(X, a, Y, b):

        constraints = {}

        for neq in neqs:
            c1, c2 = neq[0], neq[1]
            constraints[(c1,c2)] = gt
            constraints[(c2,c1)] = lt

        if X in variables and Y in variables and (X,Y) not in constraints.keys():
            constraints[(X,Y)] = diff
    
        return constraints[(X,Y)](a,b)
    
    return CSP(variables, domains, neighbors, constraints)

### Visualização do problema (?)

In [58]:
def list_coords(tuple):
    x, y = tuple[0], tuple[1]
    return ((x-1)*2, (y-1)*2)

def neq_placement(x1, y1, x2, y2):
    '''Returns the appropriate symbol to show in the output 
    along with the list coordinates (xs, ys) where it should be put.'''
    if x1 == x2:
        xs = x1
        ys = (y1 + y2) / 2
        
        if y1 > y2:
            symbol = '^'
        else: symbol = 'V'
    
    else: # y1 == y2
        xs = (x1 + x2) / 2
        ys = y1

        if x1 > x2:
            symbol = ' < '
        else: symbol = ' > '
    return (symbol, int(xs), int(ys))

def board_to_list(dim, board):
    board = force_ordered_board(dim, board)
    line_list = [[] for i in range(2*dim)]
    empty_line = ['   ' for i in range(2*dim-1)] # ['   ', '   ', ...]

    current_line = 0

    for line in line_list:
        row_from_line = current_line / 2 + 1

        if current_line % 2 == 0: # even line
            for cell,value in board.items():
                cell_row = cell[1]

                if cell_row == row_from_line:
                    line_list[current_line].append(str(value))
                    line_list[current_line].append('   ')
            line_list[current_line].pop() # remove last element, which is an empty space
            current_line += 1

        else: # odd line, must be empty
            line_list[current_line] = empty_line.copy() 
            current_line += 1

    line_list.pop() # remove last line, which is an empty line
    return line_list

def futoshiki_output(dim, board, neqs):

    line_list = board_to_list(dim, board)

    # Insert neqs
    for neq in neqs:
        c1, c2 = neq[0], neq[1]
        x1, y1 = list_coords(c1)
        x2, y2 = list_coords(c2)

        symbol, xs, ys = neq_placement(x1, y1, x2, y2)
        line_list[ys][xs] = symbol

    # Output
    for line in line_list:
        line_output = ''
        for char in line:
            line_output += char
        print(line_output)

### Instâncias e Testes

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

In [57]:
# código 

tls = futoshiki(5, numbers={(3,2):4}, neqs=[((1,1),(1,2)),((1,1),(2,1)),((1,3),(1,2))])

start = timeit.default_timer()
ls_no_inference = backtracking_search(tls, select_unassigned_variable=mrv)
stop = timeit.default_timer()
time = stop-start

print("Sem inferência:\n")
display(5, ls_no_inference, [((1,1),(1,2)),((1,1),(2,1)),((1,3),(1,2))])
print()
print(time)


Sem inferência:

5 > 1   2   3   4
V                        
2   5   4   1   3
^                        
3   4   1   2   5
                           
1   3   5   4   2
                           
4   2   3   5   1

0.06844099999989339


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

Teste 3 - Com inferência MAC


In [98]:
# Com inferência MAC
neqs = [((1, 1), (1, 2)),
        ((1, 1), (2, 1)), 
        ((1, 3), (1, 2))]
tls = futoshiki(12, numbers={(3, 2): 4}, neqs=neqs)

start = timeit.default_timer()
ls_mac = backtracking_search(tls, inference=mac)
stop = timeit.default_timer()
time = stop-start

print("\nCom inferência MAC:")
display(7, ls_mac, neqs = neqs)
print(time)



Com inferência MAC:
2 > 1   3   4   5   6   7
V                                    
1   2   4   3   6   5   8
^                                    
3   4   1   2   7   8   5
                                       
4   3   2   1   8   7   6
                                       
5   6   7   8   9   10   11
                                       
6   5   8   7   10   12   9
                                       
7   8   5   6   12   11   10
37.26268540000001


Apartir de "dim = 12" o tempo supera o 1 minuto:


In [104]:
# Com inferência MAC

neqs = [((1, 1), (1, 2)),
        ((1, 1), (2, 1)),
        ((1, 3), (1, 2))]
tls = futoshiki(12, numbers={(3, 2): 4}, neqs=neqs)

start = timeit.default_timer()
ls_mac = backtracking_search(tls, inference=mac)
stop = timeit.default_timer()
time = stop-start

print("\nCom inferência MAC:")
display(12, ls_mac, neqs = neqs)
print(time)



Com inferência MAC:
2 > 1   3   4   5   6   7   8   9   10   11   12
V                                                                  
1   2   4   3   6   5   8   7   10   9   12   11
^                                                                  
3   4   1   2   7   8   5   6   11   12   9   10
                                                                     
4   3   2   1   8   7   6   5   12   11   10   9
                                                                     
5   6   7   8   9   10   11   12   1   2   3   4
                                                                     
6   5   8   7   10   12   9   11   3   1   4   2
                                                                     
7   8   5   6   12   11   10   9   2   4   1   3
                                                                     
8   7   6   5   11   9   12   10   4   3   2   1
                                                                     
9   10   11   12   3   4   1   

Teste 4 - Com Heuristica

In [111]:
# Heurística

neqs = [((1, 1), (1, 2)),
        ((1, 1), (2, 1)),
        ((1, 3), (1, 2))]
tls = futoshiki(5, numbers={(3, 2): 4}, neqs=neqs)

start = timeit.default_timer()
ls_heu = backtracking_search(tls, select_unassigned_variable=mrv)
stop = timeit.default_timer()
time = stop-start

print("Com heurística:")
display(5, ls_heu, neqs=neqs)
print(time)


Com heurística:
5 > 3   2   4   1
V                        
1   5   4   3   2
^                        
2   4   1   5   3
                           
4   2   3   1   5
                           
3   1   5   2   4
0.007128199999897333


A partir de "dim = 8" o tempo supera o 1 minuto.

In [124]:
# Heurística

neqs = [((1, 1), (1, 2)),
        ((1, 1), (2, 1)),
        ((1, 3), (1, 2))]
tls = futoshiki(8, numbers={(3, 2): 4}, neqs=neqs)

start = timeit.default_timer()
ls_heu = backtracking_search(tls, select_unassigned_variable=mrv)
stop = timeit.default_timer()
time = stop-start

print("Com heurística:")
display(8, ls_heu, neqs=neqs)
print(time)


Com heurística:
5 > 3   6   4   2   8   7   1
V                                          
1   8   4   7   3   2   6   5
^                                          
7   6   2   1   4   3   5   8
                                             
8   4   7   2   5   1   3   6
                                             
6   1   8   5   7   4   2   3
                                             
4   2   5   3   1   6   8   7
                                             
3   5   1   6   8   7   4   2
                                             
2   7   3   8   6   5   1   4
38.41151449999961


In [55]:
print("Dominios:", tls.domains)
print()
print("Variaveis:", tls.variables)
print()
print("Vizinhos:", tls.neighbors)

Dominios: {(1, 1): [1, 2, 3, 4, 5], (1, 2): [1, 2, 3, 4, 5], (1, 3): [1, 2, 3, 4, 5], (1, 4): [1, 2, 3, 4, 5], (1, 5): [1, 2, 3, 4, 5], (2, 1): [1, 2, 3, 4, 5], (2, 2): [1, 2, 3, 4, 5], (2, 3): [1, 2, 3, 4, 5], (2, 4): [1, 2, 3, 4, 5], (2, 5): [1, 2, 3, 4, 5], (3, 1): [1, 2, 3, 4, 5], (3, 2): [4], (3, 3): [1, 2, 3, 4, 5], (3, 4): [1, 2, 3, 4, 5], (3, 5): [1, 2, 3, 4, 5], (4, 1): [1, 2, 3, 4, 5], (4, 2): [1, 2, 3, 4, 5], (4, 3): [1, 2, 3, 4, 5], (4, 4): [1, 2, 3, 4, 5], (4, 5): [1, 2, 3, 4, 5], (5, 1): [1, 2, 3, 4, 5], (5, 2): [1, 2, 3, 4, 5], (5, 3): [1, 2, 3, 4, 5], (5, 4): [1, 2, 3, 4, 5], (5, 5): [1, 2, 3, 4, 5]}

Variaveis: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]

Vizinhos: {(1, 1): [(1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (3, 1), (4, 1), (5, 1)], (1, 2): [(1, 1), (1, 3), (1, 4), (1, 5), (2, 2), (3, 2), (4, 2), (5, 2)], (1,